In [1]:
import pandas as pd
import sqlite3

### Create Python Dictionaries to store data to be inserted into DataFrames.

In [2]:
sales_data = {
    'customer_id':('A','A','A','A','A','A','B','B','B','B','B','B','C','C','C'),
    'order_date':('2021-01-01','2021-01-01','2021-01-07','2021-01-10','2021-01-11','2021-01-11',
                 '2021-01-01','2021-01-02','2021-01-04','2021-01-11','2021-01-16','2021-02-01',
                 '2021-01-01','2021-01-01','2021-01-07'),
    'product_id':(1,2,2,3,3,3,2,2,1,1,3,3,3,3,3)
}

menu_data = {
    'product_id':(1,2,3),
    'product_name':('sushi','curry','ramen'),
    'price': (10,15,12)
}

member_data = {
    'customer_id':('A','B'),
    'join_date':('2021-01-07', '2021-01-09')
}

### Create sqlite3 database, create sales, menu and member table and insert values into them.

In [3]:
conn = sqlite3.connect('dannys_diner.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS sales (
       customer_id  text,
       order_date text,
       product_id integer
       )''')

cursor.execute('''INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');

        ''')

cursor.execute('''CREATE TABLE IF NOT EXISTS menu (
       product_id  integer,
       product_name text,
       price integer
       )
       ''')

cursor.execute('''INSERT INTO menu
   ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');
        ''')


cursor.execute('''CREATE TABLE IF NOT EXISTS members (
       customer_id  text,
       join_date text
       )
       ''')

cursor.execute('''INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');
      ''');

conn.commit()

### Create DataFrames using dictionaries from above.
###### (A DataFrame is similar to an Excel or SQL table with rows and columns)

In [4]:
sales_df = pd.DataFrame(sales_data)
menu_df = pd.DataFrame(menu_data)
member_df = pd.DataFrame(member_data)

### Change format of date columns from string to datetime

In [5]:
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'])
member_df['join_date'] = pd.to_datetime(member_df['join_date'])

### Preview each DataFrame that'll be used to answer the questions below.

In [6]:
menu_df

,product_id,product_name,price
0,1,sushi,10
1,2,curry,15
2,3,ramen,12


In [7]:
sales_df

,customer_id,order_date,product_id
0,A,2021-01-01,1
1,A,2021-01-01,2
2,A,2021-01-07,2
3,A,2021-01-10,3
4,A,2021-01-11,3
5,A,2021-01-11,3
6,B,2021-01-01,2
7,B,2021-01-02,2
8,B,2021-01-04,1
9,B,2021-01-11,1


In [8]:
member_df

,customer_id,join_date
0,A,2021-01-07
1,B,2021-01-09


### 1. What is the total amount each customer spent at the restaurant?

###### --Python

In [9]:
sales_df.merge(menu_df).drop('product_id',axis=1).groupby(['customer_id']).sum().rename(columns={'price':'total_spent'})

,total_spent
customer_id,
A,76
B,74
C,36


###### --SQL

In [10]:
pd.read_sql('''
SELECT customer_id, SUM(menu.price) AS 'total_spent'
FROM sales
JOIN menu
USING(product_id)
GROUP BY customer_id
''', conn)

,customer_id,total_spent
0,A,76
1,B,74
2,C,36


### 2. How many days has each customer visted the restaurant?

###### --Python

In [11]:
sales_df.drop('product_id',axis=1).groupby(['customer_id'])\
    .agg({'order_date': lambda x: len(pd.unique(x))}).rename(columns={'order_date':'num_days_visited'})

,num_days_visited
customer_id,
A,4
B,6
C,2


###### --SQL

In [12]:
pd.read_sql('''
SELECT customer_id, COUNT(DISTINCT order_date) AS 'num_days_visited'
FROM sales
GROUP BY customer_id
''', conn)

,customer_id,num_days_visited
0,A,4
1,B,6
2,C,2


### 3. What was the first item from the menu purchased by each customer?

###### --Python

In [13]:
merge_df = sales_df.merge(menu_df) #join sales_df and menu_df on product_id
merge_df

,customer_id,order_date,product_id,product_name,price
0,A,2021-01-01,1,sushi,10
1,B,2021-01-04,1,sushi,10
2,B,2021-01-11,1,sushi,10
3,A,2021-01-01,2,curry,15
4,A,2021-01-07,2,curry,15
5,B,2021-01-01,2,curry,15
6,B,2021-01-02,2,curry,15
7,A,2021-01-10,3,ramen,12
8,A,2021-01-11,3,ramen,12
9,A,2021-01-11,3,ramen,12


In [14]:
#filter for records where 'order_date' is the smallest i.e. 'first date'
merge_df[['customer_id','order_date','product_name']].loc[merge_df['order_date'] == merge_df['order_date'].min()]

,customer_id,order_date,product_name
0,A,2021-01-01,sushi
3,A,2021-01-01,curry
5,B,2021-01-01,curry
12,C,2021-01-01,ramen
13,C,2021-01-01,ramen


###### --SQL

In [15]:
pd.read_sql('''
SELECT sub.customer_id, sub.first_date, product_name
FROM 
    (SELECT customer_id, MIN(order_date) AS 'first_date'
    FROM sales
    GROUP BY customer_id
    ORDER BY customer_id) AS sub,
    sales
    
JOIN menu
USING(product_id)
WHERE sales.customer_id = sub.customer_id AND
        sales.order_date = sub.first_date
ORDER BY sales.customer_id
''', conn)

,customer_id,first_date,product_name
0,A,2021-01-01,sushi
1,A,2021-01-01,curry
2,B,2021-01-01,curry
3,C,2021-01-01,ramen
4,C,2021-01-01,ramen


### 4. What is the most purchased item on the menu and how many times was it purchased by all the customers?

###### --Python

In [16]:
#using merge_df from above
merge_df.groupby(['product_name']).agg({'product_name':['count']}).apply(lambda x: x.sort_values(ascending=False))

,product_name
,count
product_name,
ramen,8
curry,4
sushi,3


###### --SQL

In [17]:
pd.read_sql('''
SELECT product_name, COUNT(product_name) AS 'product_count'
FROM sales
JOIN menu
USING(product_id)
GROUP BY product_name
ORDER BY product_count DESC
''', conn)

,product_name,product_count
0,ramen,8
1,curry,4
2,sushi,3


### 5. Which item was the most popular for each customer?

###### --Python

In [18]:
merge_df.groupby(['customer_id','product_name'])['product_name']\
    .agg(['count']).sort_values(by=['customer_id','count'],ascending=[True,False])

count
customer_id product_name       
A           ramen             3
            curry             2
            sushi             1
B           curry             2
            ramen             2
            sushi             2
C           ramen             3

###### --SQL

In [19]:
pd.read_sql('''
SELECT customer_id, product_name, COUNT(product_name) AS 'count'
FROM sales
JOIN menu
USING(product_id)
GROUP BY customer_id, product_name
ORDER BY customer_id, count DESC
''',conn)

,customer_id,product_name,count
0,A,ramen,3
1,A,curry,2
2,A,sushi,1
3,B,sushi,2
4,B,ramen,2
5,B,curry,2
6,C,ramen,3


### 6. Which item was purchased first by the customer after they became a member?

###### --Python

In [20]:
#perfrom inner join on member_df with merge df and store into new 'join_df'
join_df = member_df.merge(merge_df)
join_df

,customer_id,join_date,order_date,product_id,product_name,price
0,A,2021-01-07,2021-01-01,1,sushi,10
1,A,2021-01-07,2021-01-01,2,curry,15
2,A,2021-01-07,2021-01-07,2,curry,15
3,A,2021-01-07,2021-01-10,3,ramen,12
4,A,2021-01-07,2021-01-11,3,ramen,12
5,A,2021-01-07,2021-01-11,3,ramen,12
6,B,2021-01-09,2021-01-04,1,sushi,10
7,B,2021-01-09,2021-01-11,1,sushi,10
8,B,2021-01-09,2021-01-01,2,curry,15
9,B,2021-01-09,2021-01-02,2,curry,15


In [21]:
#create a after_mbr dataframe by filtering where order_date is greater than join_date and taking the lowest date
after_mbr_df = join_df[join_df['order_date'] > join_df['join_date']].groupby(['customer_id','join_date'])['order_date']\
    .agg(['min']).rename(columns={'min':'first_purchase_date_after_member'}).reset_index()
after_mbr_df

,customer_id,join_date,first_purchase_date_after_member
0,A,2021-01-07,2021-01-10
1,B,2021-01-09,2021-01-11


In [22]:
#perform a left join to obtain product_name of the first purchased item after becoming a member
after_mbr_df.merge\
    (join_df,how='left', left_on=['customer_id','first_purchase_date_after_member', 'join_date'],
     right_on=['customer_id','order_date','join_date'])\
    [['customer_id','join_date','first_purchase_date_after_member','product_name']]

,customer_id,join_date,first_purchase_date_after_member,product_name
0,A,2021-01-07,2021-01-10,ramen
1,B,2021-01-09,2021-01-11,sushi


###### --SQL

### 7. Which item was purchased just before the customer became a member?

###### --Python

In [23]:
#create a before_mbr dataframe by filtering where order_date is less than join_date and taking the highest date
before_mbr_df = join_df[join_df['order_date'] < join_df['join_date']].groupby(['customer_id','join_date'])['order_date']\
    .agg(['max']).rename(columns={'max':'last_purchase_date_before_member'}).reset_index()
before_mbr_df

,customer_id,join_date,last_purchase_date_before_member
0,A,2021-01-07,2021-01-01
1,B,2021-01-09,2021-01-04


In [24]:
#perform a left join to obtain product_name of the last purchased item before becoming a member
before_mbr_df.merge\
    (join_df,how='left', left_on=['customer_id','last_purchase_date_before_member', 'join_date'],
     right_on=['customer_id','order_date','join_date'])\
    [['customer_id','join_date','last_purchase_date_before_member','product_name']]

,customer_id,join_date,last_purchase_date_before_member,product_name
0,A,2021-01-07,2021-01-01,sushi
1,A,2021-01-07,2021-01-01,curry
2,B,2021-01-09,2021-01-04,sushi


###### --SQL

### 8. What are the total items and amount spent for each member before they became a member?

###### --Python

In [25]:
#perform a left join to obtain total amount of items and total spent
before_mbr_df.merge\
    (join_df, how='left', left_on=['customer_id','join_date','last_purchase_date_before_member'],
    right_on=['customer_id', 'join_date','order_date'])\
    [['customer_id','join_date','last_purchase_date_before_member','product_name','price']]\
    .groupby(['customer_id','last_purchase_date_before_member']).agg({'product_name':'count','price':'sum'})\
    .rename(columns={'product_name':'total_items','price':'total_spent'})

,,total_items,total_spent
customer_id,last_purchase_date_before_member,,
A,2021-01-01,2,25
B,2021-01-04,1,10


###### --SQL

### 9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

###### --Python

In [26]:
#create a points column where values are contingent upon product_name not equal to sushi

merge_df['points'] = (merge_df['price']*10).where((merge_df['product_name'] != 'sushi'), merge_df['price']*20)
merge_df.sort_values('customer_id')

,customer_id,order_date,product_id,product_name,price,points
0,A,2021-01-01,1,sushi,10,200
3,A,2021-01-01,2,curry,15,150
4,A,2021-01-07,2,curry,15,150
7,A,2021-01-10,3,ramen,12,120
8,A,2021-01-11,3,ramen,12,120
9,A,2021-01-11,3,ramen,12,120
1,B,2021-01-04,1,sushi,10,200
2,B,2021-01-11,1,sushi,10,200
5,B,2021-01-01,2,curry,15,150
6,B,2021-01-02,2,curry,15,150


In [27]:
merge_df.groupby('customer_id')['points'].agg(['sum']).rename(columns={'sum':'total_points'})

,total_points
customer_id,
A,860
B,940
C,360


###### --SQL

### 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

###### --Python

In [28]:
#create a points column where values are contingent upon boolean clauses
join_df['points'] = (join_df['price']*20).where((join_df['product_name'] == 'sushi')\
                                            |(join_df['order_date'] >= join_df['join_date'])\
                                            &(join_df['order_date'] < join_df['join_date']+ pd.Timedelta(7, unit='D'))
                                            ,join_df['price']*10)
join_df.sort_values('customer_id')

,customer_id,join_date,order_date,product_id,product_name,price,points
0,A,2021-01-07,2021-01-01,1,sushi,10,200
1,A,2021-01-07,2021-01-01,2,curry,15,150
2,A,2021-01-07,2021-01-07,2,curry,15,300
3,A,2021-01-07,2021-01-10,3,ramen,12,240
4,A,2021-01-07,2021-01-11,3,ramen,12,240
5,A,2021-01-07,2021-01-11,3,ramen,12,240
6,B,2021-01-09,2021-01-04,1,sushi,10,200
7,B,2021-01-09,2021-01-11,1,sushi,10,200
8,B,2021-01-09,2021-01-01,2,curry,15,150
9,B,2021-01-09,2021-01-02,2,curry,15,150


In [29]:
#filter for the month of January, group by customer_id and aggregate points by total sum
join_df.loc[join_df['order_date'].dt.month == 1].groupby('customer_id')\
    ['points'].agg(['sum']).rename(columns={'sum':'total_points'})

,total_points
customer_id,
A,1370
B,820


###### --SQL